In [ ]:
from sklearn.linear_model import RidgeCV #Vajalike teekide importimine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os

# Funktsioon sobivate kantregressioonimudelite koostamiseks ja tulemuste arvutamiseks kokkuvõtliku andmetabeli numbriliste tunnustel
# Param faili_tee_andmed - Etteantud tee lendude kokkuvõtlikusse andmetabelisse
# Param maatriksi_tee - Etteantud tee kausta, kuhu salvestatakse mudelite tulemuste põhjal koostatud koefitsentide maatriks
# Param maatriksi_nimi - Failinimi maatriksi salvestamiseks kausta
# Param test_suurus - osakaal andmestikust, mis eraldatakse loodud mudeli hindamiseks ja täpsuse arvutamiseks. Vaikimisi suurus 0.2
# Param max_random_state - parameeter arvude vahemikuga, kontrollimaks juhuslikkust parima mudeli valimiseks, et mudeli treenimine saaks olle hiljem reprodutseeritav. Vaikimisi vahemikus 0-250.
# alpha_väärtused - scikit-learn RidgeCV funktsiooni karistusparameeter λ. Vaikimisi väärtus None ja sobivad väärtused on programmi sisse kirjutatud, mida testitakse. 
# Väljastatakse iga sõltumatu tunnuse mõju (koefitsent) sõltuvale tunnusele. Salvestatakse saadud koefitsentide maatriks. 

def regressioon_ridge_standardiseeritud(faili_tee_andmed, maatriksi_tee, maatriksi_nimi, test_suurus=0.2, max_random_state=250, alpha_väärtused=None):
    df = pd.read_csv(faili_tee_andmed) # Andmete sisse lugemine

    tulbad = ["Liiklusinfo", "T", "η (seansse)", "тm", "Teisi õ/s"] # Numbriliste väärtustega tunnustega tulpade valimine
    df[tulbad] = df[tulbad].apply(pd.to_numeric, errors="coerce") # Väärtuste muutmine numbriliseks väärtuseks
    df = df.dropna(subset=tulbad) # Andmeridade välja selekteerimine, kus vähemalt üks tunnus on puudu
    df["T"] = df["T"] / 60 # Sagedusel oldud aja teisendamine minutiteks

    tunnused = ["T", "Liiklusinfo", "η (seansse)", "тm", "Teisi õ/s"] # Tunnuste nimetamine maatriksi koostamise jaoks

    if alpha_väärtused is None: # Karistusparameeterite loomine, kui sisend on None
        alpha_väärtused = np.logspace(-4, 4, 20)

    maatriks_andmed = pd.DataFrame(index=tunnused, columns=[t for t in tunnused]) # Maatriksi koostamiseks andmetabeli koostamine

    for sõltuv_tunnus in tunnused: # Tsükliga valitakse iga tunnus sõltuvaks tunnuseks, ning vastava regressiooni parameetrid
        parim_skoor = float("-inf") # Parima mudeli skoor testgrupile muutuja initsialiseerimine
        parim_state = None # Parima mudeli 'random state' muutuja initsialiseerimine
        parim_tunnused = None # Parima mudeli tunnuste muutuja initsialiseerimine
        parim_treening_skoor = None # Muutuja initsialiseerimine parima mudeli treeningrupi andmete skoori salvestamiseks
        parim_koef = None # Parima mudeli koefitsentide muutuja initsialiseerimine
        parim_alpha = None # Parima mudeli karistusparameeteri muutuja initsialiseerimine

        tunnused_stds = df[[tunnus for tunnus in tunnused if tunnus != sõltuv_tunnus]].std().values # Iga tunnuse standardhälbe leidmine, kuna regressioonimudelite koefitsendid näitavad sõltuva tunnuse muutu, 
                                                                                                    # kui sõltumatut tunnust suurendada ühe tema standardhälbe võrra. Teades standardhälbeid, on võimalik saadud 
                                                                                                    # tulemused teisendada tagasi algühikuteks

        for state in range(max_random_state): # Parima mudeli leidmiseks käiakse läbi kõik 'random_state' vahemikku jäävad täisarvud
            X = df[[tunnus for tunnus in tunnused if tunnus != sõltuv_tunnus]] # Andmestiku jagamine sõltuvaks ja sõltumatuteks tunnusteks
            y = df[sõltuv_tunnus]

            X_train, X_test, y_train, y_test = train_test_split( #Andmestike jagamine juhuslikuse alusel treening ja testgrupiks
                X, y, test_size=test_suurus, random_state=state
            )

            scaler = StandardScaler() # Sõltumatute tunnuste standardiseerimine
            X_train_scaled = scaler.fit_transform(X_train)
            X_test_scaled = scaler.transform(X_test)

            mudel = RidgeCV(alphas=alpha_väärtused) # Kantregressioonimudeli treenimine
            mudel.fit(X_train_scaled, y_train)
            skoor = mudel.score(X_test_scaled, y_test) # Skoori arvutamine testandmetel

            if skoor > parim_skoor: # Kui skoor on parem kui eelmine parim, kirjutatakse järgnevad muutujad uute tulemustega üle
                parim_skoor = skoor
                parim_state = state
                parim_alpha = mudel.alpha_
                parim_tunnused = X.columns
                parim_treening_skoor = mudel.score(X_train_scaled, y_train)
                parim_koef = mudel.coef_

        print(f"Parim kantregressioon sõltuva tunnusega '{sõltuv_tunnus}':") # Tulemuste väljastamine ekraanile
        print(f"  Parim 'random state': {parim_state}, parim alpha: {parim_alpha}")
        print(f"  R^2 skoor (treening): {parim_treening_skoor:.4f}, R^2 skoor (test):  {parim_skoor:.4f}")
        print(f"  Koefitsendid:")

        for tunnus, koef, std in zip(parim_tunnused, parim_koef, tunnused_stds):
            mõju_algühikutes = koef / std
            print(f"    {tunnus} → {sõltuv_tunnus}:")
            print(f"      Koefitsient (muutus '{sõltuv_tunnus}' originaal ühikutes ühe '{tunnus}' standardhälbe (std={std:.4f}) tõusu kohta): {koef:.4f}")
            print(f"      Muutus '{sõltuv_tunnus}' algühikutes ühe ühiku tõusu korral '{tunnus}': {mõju_algühikutes:.4f} ühikut")

            maatriks_andmed.loc[sõltuv_tunnus, tunnus] = koef # Tulemuste salvestamine maatriksi jaoks andmetabelisse

        print("-" * 50)

    plot_maatriks = maatriks_andmed.copy() # Regressioonimudelite koefitsentimaatriksi koostamine

    plot_maatriks.rename(index={"η (seansse)": "η"}, inplace=True)
    plot_maatriks.rename(columns={"η (seansse)": "η"}, inplace=True)

    plt.figure(figsize=(10, 6))
    ax = sns.heatmap(plot_maatriks.astype(float), annot=True, fmt=".2f", cmap="coolwarm", center=0)

    ax.set_xlabel("Sõltumatu tunnus", fontname="Times New Roman", fontsize=14, color="black")
    ax.set_ylabel("Sõltuv tunnus", fontname="Times New Roman", fontsize=14, color="black")

    ax.tick_params(axis='x', rotation=0, colors='black')
    for tick in ax.get_xticklabels():
        tick.set_fontname("Times New Roman")
        tick.set_color("black")
        tick.set_fontsize(12)

    ax.tick_params(axis='y', colors='black')
    for tick in ax.get_yticklabels():
        tick.set_fontname("Times New Roman")
        tick.set_color("black")
        tick.set_fontsize(12)

    cbar = ax.collections[0].colorbar
    if cbar:
        cbar.ax.yaxis.set_tick_params(color='black')
        cbar.ax.yaxis.label.set_color('black')
        for label in cbar.ax.get_yticklabels():
            label.set_fontname("Times New Roman")
            label.set_color("black")
            label.set_fontsize(12)

    plt.tight_layout()

    pildi_tee_väljund = os.path.join(maatriksi_tee, maatriksi_nimi)
    plt.savefig(pildi_tee_väljund, format="svg")
    plt.close()